# Hello, today I'm going to go through a used car dataset downloaded from Kaggle. I'm going to be cleaning and modifying the data so that I can use it for data visualization.

In [2]:
import pandas as pd
import numpy as np

In [3]:
import xlsxwriter
import openpyxl

## Below, I load the dataset using the pandas library.

In [4]:
df = pd.read_csv(r'C:\Users\wbouley\Desktop\Portfolio\Other_CSVs\car_price.csv')

In [5]:
df.head()

,Unnamed: 0,car_name,car_prices_in_rupee,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats
0,0,Jeep Compass 2.0 Longitude Option BSIV,10.03 Lakh,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats
1,1,Renault Duster RXZ Turbo CVT,12.83 Lakh,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats
2,2,Toyota Camry 2.5 G,16.40 Lakh,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats
3,3,Honda Jazz VX CVT,7.77 Lakh,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats
4,4,Volkswagen Polo 1.2 MPI Highline,5.15 Lakh,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats


## Now, I'm going to check the data types of each column to see what I'm working with.

In [6]:
df.dtypes

Unnamed: 0              int64
car_name               object
car_prices_in_rupee    object
kms_driven             object
fuel_type              object
transmission           object
ownership              object
manufacture             int64
engine                 object
Seats                  object
dtype: object

## Since the currency is in Rupee's ("INR"), and I am most familiar with USD, I need to create a function that will convert the currency to USD.

In [7]:
'''Function to convert rupees to USD'''

def convert_to_usd(value):
    # Remove the ' Lakh' suffix from the value
    value = value[:-5]
    # Convert the value to a float
    value = float(value)
    # Convert the value from rupees to USD using the current exchange rate
    value = (value*100000)*0.012

    return value

values = df['car_prices_in_rupee']
usd_values = [convert_to_usd(value) for value in values]

## After the function is defined and run, I need to create a column that lists the prices of each  car in each row of the dataset. I do use by using the code in the cell below. Then, I check to see if the column was created by calling the df.columns method.

In [8]:
df['car_price_usd'] = usd_values

In [9]:
df.columns

Index(['Unnamed: 0', 'car_name', 'car_prices_in_rupee', 'kms_driven',
       'fuel_type', 'transmission', 'ownership', 'manufacture', 'engine',
       'Seats', 'car_price_usd'],
      dtype='object')

## I see that I now have another column that I don't want: 'Unnamed: 0'. I'll call the df.drop() method to remove that column. I don't want to create a new dataframe so I set inplace=True. Axis=1 drops the column while axis=0 means drop the row.

In [10]:
df.drop(['Unnamed: 0', 'car_prices_in_rupee'], axis=1, inplace=True)

In [11]:
df.columns

Index(['car_name', 'kms_driven', 'fuel_type', 'transmission', 'ownership',
       'manufacture', 'engine', 'Seats', 'car_price_usd'],
      dtype='object')

In [12]:
df.head()

,car_name,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats,car_price_usd
0,Jeep Compass 2.0 Longitude Option BSIV,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats,12036.0
1,Renault Duster RXZ Turbo CVT,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats,15396.0
2,Toyota Camry 2.5 G,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats,19680.0
3,Honda Jazz VX CVT,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats,9324.0
4,Volkswagen Polo 1.2 MPI Highline,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats,6180.0


## I see that I have a column in the dataset that uses kilometers as a measurement. I need to convert this to miles. I do so by using the below code.

In [13]:
df['kms_driven'] = df['kms_driven'].str.replace(',', '')

In [14]:
def convert_to_miles(value):
    # Remove the ' Lakh' suffix from the value
    value = value[:-4]
    # Convert the value to a float
    value = float(value)
    # Convert the value from rupees to USD using the current exchange rate
    value = value*(1.60934-1)

    return value

values = df['kms_driven']
miles_driven = [convert_to_miles(value) for value in values]

df['miles_driven'] = miles_driven

## After I converted the kilometers column to a new miles column, I need to remove the df['kms_driven'] column. I use the df.drop() method similar to what I did before. Once again, I use the inplace=True and axis=1 arguments to keep the same dataframe and drop the df['kms_driven'] column.

In [15]:
df.drop('kms_driven', axis=1, inplace=True)

In [16]:
df.head()

,car_name,fuel_type,transmission,ownership,manufacture,engine,Seats,car_price_usd,miles_driven
0,Jeep Compass 2.0 Longitude Option BSIV,Diesel,Manual,1st Owner,2017,1956 cc,5 Seats,12036.0,52540.95084
1,Renault Duster RXZ Turbo CVT,Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats,15396.0,8072.53632
2,Toyota Camry 2.5 G,Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats,19680.0,36769.40362
3,Honda Jazz VX CVT,Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats,9324.0,16266.94064
4,Volkswagen Polo 1.2 MPI Highline,Petrol,Manual,1st Owner,2016,1199 cc,5 Seats,6180.0,42296.72676


## I don't like the 'Petrol' fuel type, so I'm going to change it to Gasoline. Below, I use a lambda function to change every instance of Petrol in the dataframe to Gasoline.

In [17]:
df['fuel_type'].unique()

array(['Diesel', 'Petrol', 'Cng', 'Electric', 'Lpg'], dtype=object)

In [18]:
df['fuel_type'] = df['fuel_type'].apply(lambda x: 'Gasoline' if x == 'Petrol' else x)

In [19]:
df.head()

,car_name,fuel_type,transmission,ownership,manufacture,engine,Seats,car_price_usd,miles_driven
0,Jeep Compass 2.0 Longitude Option BSIV,Diesel,Manual,1st Owner,2017,1956 cc,5 Seats,12036.0,52540.95084
1,Renault Duster RXZ Turbo CVT,Gasoline,Automatic,1st Owner,2021,1330 cc,5 Seats,15396.0,8072.53632
2,Toyota Camry 2.5 G,Gasoline,Automatic,1st Owner,2016,2494 cc,5 Seats,19680.0,36769.40362
3,Honda Jazz VX CVT,Gasoline,Automatic,1st Owner,2018,1199 cc,5 Seats,9324.0,16266.94064
4,Volkswagen Polo 1.2 MPI Highline,Gasoline,Manual,1st Owner,2016,1199 cc,5 Seats,6180.0,42296.72676


## I keep using the df.head() method to verify that the changes have been made correctly. Now I would like to rename a couple of columns to make it a little bit cleaner. I use the df.rename() method to create a new dataframe.

In [20]:
df = df.rename(columns={'manufacture': 'year', 'Seats': 'seats'})

In [21]:
df.head()

,car_name,fuel_type,transmission,ownership,year,engine,seats,car_price_usd,miles_driven
0,Jeep Compass 2.0 Longitude Option BSIV,Diesel,Manual,1st Owner,2017,1956 cc,5 Seats,12036.0,52540.95084
1,Renault Duster RXZ Turbo CVT,Gasoline,Automatic,1st Owner,2021,1330 cc,5 Seats,15396.0,8072.53632
2,Toyota Camry 2.5 G,Gasoline,Automatic,1st Owner,2016,2494 cc,5 Seats,19680.0,36769.40362
3,Honda Jazz VX CVT,Gasoline,Automatic,1st Owner,2018,1199 cc,5 Seats,9324.0,16266.94064
4,Volkswagen Polo 1.2 MPI Highline,Gasoline,Manual,1st Owner,2016,1199 cc,5 Seats,6180.0,42296.72676


## I'm finnaly happy with how the data looks. Finally, I want to save this data into an excel file so that I can upload it to Tableau to do some nice visualizations. I do so by using the following code.

In [23]:
excel_file = r'C:\Users\wbouley\Desktop\Portfolio\Excel Files\used_car_data.xlsx'
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
writer.close()

In [24]:
df.to_excel(excel_file, index=False)

## Thanks for following along! I really appreciate it!


## Here is a link to the Tableau viz if you want to take a look at it!

[Used Car Data Viz](https://public.tableau.com/app/profile/wade.bouley/viz/UsedCarData_16723538293150/Dashboard3#1)
